# Ideas Dataset

Лаботоратория по искусственному интеллекту, Сбербанк. 

Авторы: [Борис Шминке](<mailto:Shminke.B.A@omega.sbrf.ru>), [Роза Айсина](<mailto:Aysina.R.M@omega.sbrf.ru>). 

О чем: применение рекомендательных систем к датасету историй. Схема разбиения - делим лог на две рандомные части.

## Содержание

1. [Импорты, создание спарк-сессии](#intro)
2. [Подготовка данных](#data-preparator)
3. [Рекомендатель на основе популярности](#popular-recommender)
3.1. [Запуск single модели](#popular-recommender-single)
3.2. [Подбор гиперпараметров](#popular-recommender-scenario)
4. [Рекомендатель на основе k-NN для объектов](#knn-recommender)
4.1. [Запуск single модели](#knn-recommender-single)
4.2. [Подбор гиперпараметров](#knn-recommender-scenario)

### Импорты, создание спарк-сессии <a name='intro'></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf

In [3]:
parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [4]:
from sponge_bob_magic.data_preparator.data_preparator import DataPreparator
from sponge_bob_magic.metrics.metrics import Metrics
from sponge_bob_magic.models.popular_recomennder import PopularRecommender
from sponge_bob_magic.models.knn_recommender import KNNRecommender
from sponge_bob_magic.models.neurocf_recommender import NeuroCFRecommender
from sponge_bob_magic.scenarios.knn_scenario import KNNScenario
from sponge_bob_magic.scenarios.popular_scenario import PopularScenario
from sponge_bob_magic.validation_schemes import ValidationSchemes

In [5]:
# отображение максимальной ширины колонок в pandas датафреймах
pd.options.display.max_colwidth = -1

In [6]:
spark_memory = "10g"
spark_cores = "*"
# user_home = "/Users/roseaysina"
user_home = "/home/SIGMA/14563915"

spark = (
    SparkSession
    .builder
    .config('spark.driver.memory', spark_memory)
    .config('spark.local.dir', os.path.join(user_home, "tmp"))
    .master(f'local[{spark_cores}]')
    .enableHiveSupport()
    .getOrCreate()
)

spark

In [7]:
spark_logger = logging.getLogger('py4j')
spark_logger.setLevel(logging.WARN)

In [8]:
logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s, %(name)s, %(levelname)s: %(message)s',
                              datefmt='%d-%b-%y %H:%M:%S')
hdlr = logging.StreamHandler()
hdlr.setFormatter(formatter)
logger.addHandler(hdlr)
logger.setLevel(logging.DEBUG)

## Подготовка данных <a name='data-preparator'></a>

Мердж двух датасетов за 17 и 18 год. Запускается один раз. 

In [9]:
def merge_data():
    user_item_data = (
        pd.read_csv(
            "/mnt/wind/ideas_data/2017.csv",
            sep=";",
            usecols=["id_project", "user_id", "type"],
            dtype=str
        ).append(
            pd.read_csv(
                "/mnt/wind/ideas_data/2018.csv",
                sep=";",
                usecols=["id_project", "user_id", "type"],
                dtype=str
            )
        ).query("type in ('Pim::IdeaBestPracticeVote', 'Pim::IdeaResourceVote', 'Pim::IdeaVote')")
        .drop(columns=["type"])
        .rename(columns={"user_id": "user_id", "id_project": "item_id"})
    )
    user_item_data.to_csv("data.csv", index=False)

In [10]:
path_log = "/home/SIGMA/14563915/code/sponge-bob-magic/data/ideas_data/data.csv"

In [11]:
dp = DataPreparator(spark)

In [12]:
df = dp.transform_log(
    path=path_log,
    format_type="csv",
    header=True,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id"
    }
).cache()

In [13]:
df.show(3)

In [14]:
df.count()

In [15]:
df.agg(*(sf.countDistinct(sf.col(c)).alias(c) for c in df.columns)).show()

In [16]:
df.agg(sf.min(sf.col("timestamp")), sf.max(sf.col("timestamp"))).show()

In [17]:
df_short = df.limit(1000).cache()

In [18]:
splitter = ValidationSchemes(spark)

train, test_input, test = splitter.log_split_randomly(
    df, test_size=0.2,
    drop_cold_users=True, drop_cold_items=True
)

(
    train.count(), 
    test_input.count(), 
    test.count()
)

## Рекомендатель на основе популярности (popularity based recommender) <a name='popular-recommender'></a>

### Запуск single модели <a name='popular-recommender-single'></a>

In [20]:
pr = PopularRecommender(spark, alpha=0, beta=0)

In [21]:
%%time

pr.fit(
    log=train,
    user_features=None,
    item_features=None,
    path=None
)

In [22]:
%%time

recs = pr.predict(
    k=100,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    context='no_context',
    log=train,
    user_features=None,
    item_features=None,
    to_filter_seen_items=True,
    path=None
).cache()

In [23]:
recs.show(5)

In [ ]:
%%time

metric = Metrics.hit_rate_at_k(recs, df, k=100)
metric

### Подбор гиперпараметров в популярной модели <a name='popular-recommender-scenario'></a>

Популярность объекта определяется как: 

$$ popularity(i) = \dfrac{N_i + \alpha}{N + \beta}, $$

где $ N_i $ &mdash; количество пользователей, у которых было взаимодействие с данным объектом $ i $, 
$ N $ &mdash; общее количество пользователей, которые как провзаимодействовали с объектом, так и нет,
$ \alpha, \beta \in [0, \infty) $ &mdash; параметры модели. 

Эвристика: размуным пределом для параметров $ \alpha $ и $ \beta $ может стать среднее значение количества пользователей $ N_i $, которые провзаимодействовали с объектами.

In [42]:
avg_num_users = (
    df
    .select('user_id', 'item_id')
    .groupBy('item_id')
    .count()
    .select(sf.mean(sf.col('count')).alias('mean'))
    .collect()[0]['mean']
)

avg_num_users

In [31]:
popular_scenario = PopularScenario(spark)
popular_scenario.seed = 9876

In [32]:
%%time

popular_params_grid = {
    'alpha': (0, 1), 
    'beta': (0, 1)
}
results = None

best_params = popular_scenario.research(
    popular_params_grid,
    df,
    users=None, items=None,
    user_features=None,
    item_features=None,
    test_start=None,
    test_size=0.3,
    k=15, context='no_context',
    to_filter_seen_items=True,
    n_trials=1, 
    n_jobs=1,
    how_to_split='randomly'
)

best_params

In [36]:
%%time

popular_params_grid = {
    'alpha': (80, 1000), 
    'beta': (80, 1000)
}

best_params = popular_scenario.research(
    popular_params_grid,
    df,
    users=None, items=None,
    user_features=None,
    item_features=None,
    test_start=None,
    test_size=0.3,
    k=15, context='no_context',
    to_filter_seen_items=True,
    n_trials=15, 
    n_jobs=1,
    how_to_split='randomly'
)

best_params

In [37]:
results = pd.concat([popular_scenario.study.trials_dataframe(), results], axis=0)

results

In [39]:
def plot_result_value(results):
    results['value_name'] = (results['params']['alpha'].astype(str)
                         .str.cat(results['params']['beta'].astype(str), sep=', '))
    
    plt.figure(figsize=(7, 5))
    ax = results['value'].plot(kind='bar', xticks=range(len(results)), rot=0)
    ax.set_xticklabels(results['value_name'].values)

    plt.xlabel(r'$(\alpha, \beta)$' + ' пары')
    plt.ylabel('Значение метрики')
    plt.title('Результаты эксперимента')

    plt.show()
    

plot_result_value(results)

In [40]:
%%time

best_recs = popular_scenario.production(
    best_params,
    df,
    users=None,
    items=None,
    user_features=None,
    item_features=None,
    k=10,
    context='no_context',
    to_filter_seen_items=True
)

In [41]:
best_recs.show(10)

## Рекомендатель на основе k-NN для объектов (item k-NN based recommender) <a name='knn-recommender'></a>

### Запуск single модели <a name='knn-recommender-single'></a>

In [43]:
splitter = ValidationSchemes(spark)

train, test_input, test = splitter.log_split_randomly(
    df, test_size=0.2,
    drop_cold_users=False, drop_cold_items=True
)

(
    train.count(), 
    test_input.count(), 
    test.count()
)

In [44]:
knn = KNNRecommender(spark, num_neighbours=15, shrink=0)

In [45]:
%%time

knn.fit(
    log=train,
    user_features=None,
    item_features=None,
    path=None
)

In [57]:
%%time

recs = knn.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    context='no_context',
    log=train,
    user_features=None,
    item_features=None,
    to_filter_seen_items=True,
    path=None
).cache()

In [47]:
recs.show(5)

In [58]:
%%time

metric = Metrics.hit_rate_at_k(recs, test, k=100)
metric

### Подбор гиперпараметров в item k-NN модели <a name='knn-recommender-scenario'></a>

In [49]:
knn_scenario = KNNScenario(spark)
knn_scenario.seed = 3951

In [62]:
%%time

knn_params_grid = {
    'num_neighbours': (10, 100, 5), 
    'shrink': [0]
}

best_params = knn_scenario.research(
    knn_params_grid,
    df,
    users=None, items=None,
    user_features=None,
    item_features=None,
    test_start=None,
    test_size=0.2,
    k=10, context='no_context',
    to_filter_seen_items=True,
    n_trials=15, 
    n_jobs=1,
    how_to_split='randomly'
)

In [63]:
knn_results = knn_scenario.study.trials_dataframe()

knn_results

## Рекомендатель на основе нейросетки (neural CF) <a name='neurocf-recommender'></a>

### Запуск single модели <a name='neurocf-recommender-single'></a>

In [19]:
# tmp_path = "/Users/roseaysina/code/sponge-bob-magic/data"
tmp_path = "/home/SIGMA/14563915/code/sponge-bob-magic/data"

In [20]:
ncf = NeuroCFRecommender(spark,
                         learning_rate=0.01,
                         epochs=10,
                         batch_size_fit=100000,
                         embedding_dimension=100)

In [21]:
%%time

ncf.fit(
    log=train,
    user_features=None,
    item_features=None,
    path=tmp_path
)

In [29]:
import torch
torch.cuda.empty_cache()

In [30]:
import gc
gc.collect()

In [23]:
test.select('item_id').distinct().count()

In [32]:
test.select('user_id').distinct().count()

In [31]:
%%time

recs = ncf.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    context='no_context',
    log=train,
    user_features=None,
    item_features=None,
    to_filter_seen_items=True,
    path=tmp_path
)

In [27]:
%%time

metric = Metrics.hit_rate_at_k(recs, test, k=10)
metric